# Patterns Mining & Association Rules
Mining frequent patterns and association rules over the [House prices dataset (Kaggle)](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) <br> 



### 0. Imports

In [1]:
#data processing
import pandas as pd
import numpy as np
import scipy as sp

#statistics
from scipy import stats
import statsmodels.api as sm


#data visualizations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


#Machine learning library
import sklearn

#Patterns Mining
from efficient_apriori import apriori

import warnings
warnings.filterwarnings("ignore")


##### Loading the CSV dataset

In [2]:
dataset_path = "../datasets/houseprices/train.csv"
dtf = pd.read_csv(dataset_path,index_col='Id')

##### Defining numeric and categorical columns

In [3]:
numeric_columns = dtf.dtypes[(dtf.dtypes=="float64") | (dtf.dtypes=="int64")].index.tolist()
very_numerical = [nc for nc in numeric_columns if dtf[nc].nunique()>20]
categorical_columns = [c for c in dtf.columns if c not in numeric_columns]
ordinals = list(set(numeric_columns) - set(very_numerical))

##### Filling Null Values with the column's mean

In [4]:
na_columns = dtf[very_numerical].isna().sum()
na_columns = na_columns[na_columns>0]
for nc in na_columns.index:
    dtf[nc].fillna(dtf[nc].mean(),inplace=True)

##### Dropping and filling NA values for categorical columns:

1. drop if at least 70% are NA:

In [5]:
nul_cols = dtf[categorical_columns].isna().sum()/len(dtf)
drop_us = nul_cols[nul_cols>0.7]

In [6]:
dtf=dtf.drop(drop_us.index,axis=1)
categorical_columns = list(set(categorical_columns)-set(drop_us.index))

2. Fill with a new 'na' category:

In [7]:
dtf[categorical_columns]= dtf[categorical_columns].fillna('na')

### 1. Basic Apriori Example

In [8]:
transactions = [('eggs', 'milk', 'soup'),
                ('eggs', 'milk', 'apple'),
                ('soup', 'milk', 'banana')]


In [9]:
itemsets, rules = apriori(transactions, min_support=0.5, min_confidence=1)
print(rules)  # [{eggs} -> {bacon}, {soup} -> {bacon}]

[{eggs} -> {milk}, {soup} -> {milk}]


In [10]:
r = rules[0]

In [11]:
type(r)

efficient_apriori.rules.Rule

In [12]:
r.confidence,r.lift

(1.0, 1.0)

In [13]:
r.lhs,r.rhs

(('eggs',), ('milk',))

### 2. Using Apriori on a DataFrame:

We neet to convert our dataframe to a list of transacitons </br>
*Remember:* We want to keep the column names

In [14]:
df=dtf.copy()

In [15]:
records = df[categorical_columns[:5]].to_dict(orient='records')
records[:2]

[{'Condition2': 'Norm',
  'HouseStyle': '2Story',
  'BsmtQual': 'Gd',
  'GarageType': 'Attchd',
  'Heating': 'GasA'},
 {'Condition2': 'Norm',
  'HouseStyle': '1Story',
  'BsmtQual': 'Gd',
  'GarageType': 'Attchd',
  'Heating': 'GasA'}]

In [16]:
transactions=[]
for r in records:
    transactions.append(list(r.items()))


In [17]:
transactions[:2]

[[('Condition2', 'Norm'),
  ('HouseStyle', '2Story'),
  ('BsmtQual', 'Gd'),
  ('GarageType', 'Attchd'),
  ('Heating', 'GasA')],
 [('Condition2', 'Norm'),
  ('HouseStyle', '1Story'),
  ('BsmtQual', 'Gd'),
  ('GarageType', 'Attchd'),
  ('Heating', 'GasA')]]

In [18]:
itemsets, rules = apriori(transactions, min_support=0.1, min_confidence=0.8,output_transaction_ids=False)
#print(rules)  # [{eggs} -> {bacon}, {soup} -> {bacon}]

In [19]:
len(rules)

102

In [20]:
rules[:10]


[{('BsmtQual', 'Gd')} -> {('Condition2', 'Norm')},
 {('BsmtQual', 'Gd')} -> {('Heating', 'GasA')},
 {('BsmtQual', 'TA')} -> {('Condition2', 'Norm')},
 {('BsmtQual', 'TA')} -> {('Heating', 'GasA')},
 {('GarageType', 'Attchd')} -> {('Condition2', 'Norm')},
 {('GarageType', 'Detchd')} -> {('Condition2', 'Norm')},
 {('Heating', 'GasA')} -> {('Condition2', 'Norm')},
 {('Condition2', 'Norm')} -> {('Heating', 'GasA')},
 {('HouseStyle', '1.5Fin')} -> {('Condition2', 'Norm')},
 {('HouseStyle', '1Story')} -> {('Condition2', 'Norm')}]

##### What about numerical data? We need to bin....

In [21]:
binned_price = pd.qcut(dtf.SalePrice,5,labels=["very low", "low", "medium", "high","very high"])

In [22]:
binned_price.head()

Id
1         high
2         high
3         high
4          low
5    very high
Name: SalePrice, dtype: category
Categories (5, object): ['very low' < 'low' < 'medium' < 'high' < 'very high']

Binning all numeric columns in the same manner:

In [23]:
for c in very_numerical:
    try:
        df[c] = pd.qcut(dtf[c],5,labels=["very low", "low", "medium", "high","very high"])
    except:
        #sometimes for highly skewed data, we cannot perform qcut as most quantiles are equal
        df[c] = pd.cut(dtf[c],5,labels=["very low", "low", "medium", "high","very high"])

Focusing on prominent columns:

In [24]:
good_columns = ['OverallQual', 'YearBuilt', 'YearRemodAdd', 'OverallQual', 'OverallCond', 'BldgType', 'LotArea', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'LotFrontage', 'TotalBsmtSF', 'SalePrice']
gdf = df[good_columns]

In [25]:
gdf.head()

,OverallQual,YearBuilt,YearRemodAdd,OverallQual,OverallCond,BldgType,LotArea,GrLivArea,FullBath,BedroomAbvGr,LotFrontage,TotalBsmtSF,SalePrice
Id,,,,,,,,,,,,,
1,7,high,high,7,5,1Fam,low,high,2,3,low,low,high
2,6,medium,low,6,8,1Fam,medium,low,2,3,high,high,high
3,7,high,high,7,5,1Fam,high,high,2,3,low,medium,high
4,7,very low,low,7,5,1Fam,medium,high,1,3,low,low,low
5,8,high,high,8,5,1Fam,very high,very high,2,4,very high,high,very high


Converting to transactions:

In [26]:
records = gdf.to_dict(orient='records')
transactions=[]
for r in records:
    transactions.append(list(r.items()))

In [27]:
transactions[:2]

[[('OverallQual', 7),
  ('YearBuilt', 'high'),
  ('YearRemodAdd', 'high'),
  ('OverallCond', 5),
  ('BldgType', '1Fam'),
  ('LotArea', 'low'),
  ('GrLivArea', 'high'),
  ('FullBath', 2),
  ('BedroomAbvGr', 3),
  ('LotFrontage', 'low'),
  ('TotalBsmtSF', 'low'),
  ('SalePrice', 'high')],
 [('OverallQual', 6),
  ('YearBuilt', 'medium'),
  ('YearRemodAdd', 'low'),
  ('OverallCond', 8),
  ('BldgType', '1Fam'),
  ('LotArea', 'medium'),
  ('GrLivArea', 'low'),
  ('FullBath', 2),
  ('BedroomAbvGr', 3),
  ('LotFrontage', 'high'),
  ('TotalBsmtSF', 'high'),
  ('SalePrice', 'high')]]

Rules mining process:

In [28]:
itemsets, rules = apriori(transactions, min_support=0.05, min_confidence=0.8)
len(rules)

1787

In [29]:
rules[:5]

[{('BedroomAbvGr', 3)} -> {('BldgType', '1Fam')},
 {('BedroomAbvGr', 4)} -> {('BldgType', '1Fam')},
 {('FullBath', 1)} -> {('BldgType', '1Fam')},
 {('GrLivArea', 'high')} -> {('BldgType', '1Fam')},
 {('GrLivArea', 'medium')} -> {('BldgType', '1Fam')}]

### 2. Analyzing the rules:
We need a better way to see the rules.

Let's extract all the rules attributes for all rules and put them in a dataframe:

In [30]:
attrs = [a for a in dir(rules[0]) if not a.startswith("_")]
attrs

['confidence',
 'conviction',
 'count_full',
 'count_lhs',
 'count_rhs',
 'lhs',
 'lift',
 'num_transactions',
 'rhs',
 'rpf',
 'support']

In [31]:
rules_rec = []
for r in rules:
    rdict={}
    for a in attrs:
        rdict[a]=getattr(r,a)
        rdict["rule"]=str(r).split("} (")[0]+"}"
        rdict["len_l"]=len(r.lhs)
        rdict["len_r"]=len(r.rhs)
    rules_rec.append(rdict)

In [32]:
rules_df = pd.DataFrame(rules_rec)
rules_df.set_index('rule',inplace=True)
rules_df = rules_df[['len_l', 'len_r','count_lhs','count_rhs','support','confidence','lift','rpf','conviction','lhs','rhs']]

In [33]:
rules_df

,len_l,len_r,count_lhs,count_rhs,support,confidence,lift,rpf,conviction,lhs,rhs
rule,,,,,,,,,,,
"{('BedroomAbvGr', 3)} -> {('BldgType', '1Fam')}",1,1,804,1220,0.514384,0.934080,1.117833,0.480475,2.493668,"((BedroomAbvGr, 3),)","((BldgType, 1Fam),)"
"{('BedroomAbvGr', 4)} -> {('BldgType', '1Fam')}",1,1,213,1220,0.123288,0.845070,1.011314,0.104187,1.061021,"((BedroomAbvGr, 4),)","((BldgType, 1Fam),)"
"{('FullBath', 1)} -> {('BldgType', '1Fam')}",1,1,650,1220,0.393836,0.884615,1.058638,0.348393,1.424658,"((FullBath, 1),)","((BldgType, 1Fam),)"
"{('GrLivArea', 'high')} -> {('BldgType', '1Fam')}",1,1,292,1220,0.169178,0.845890,1.012295,0.143106,1.066667,"((GrLivArea, high),)","((BldgType, 1Fam),)"
"{('GrLivArea', 'medium')} -> {('BldgType', '1Fam')}",1,1,292,1220,0.160274,0.801370,0.959016,0.128439,0.827586,"((GrLivArea, medium),)","((BldgType, 1Fam),)"
...,...,...,...,...,...,...,...,...,...,...,...
"{('BldgType', '1Fam'), ('SalePrice', 'very high'), ('TotalBsmtSF', 'very high'), ('YearRemodAdd', 'very high')} -> {('OverallCond', 5), ('YearBuilt', 'very high')}",4,2,78,271,0.050000,0.935897,5.042104,0.046795,12.704383,"((BldgType, 1Fam), (SalePrice, very high), (To...","((OverallCond, 5), (YearBuilt, very high))"
"{('BldgType', '1Fam'), ('SalePrice', 'very high'), ('TotalBsmtSF', 'very high'), ('YearBuilt', 'very high')} -> {('OverallCond', 5), ('YearRemodAdd', 'very high')}",4,2,89,186,0.050000,0.820225,6.438323,0.041011,4.853853,"((BldgType, 1Fam), (SalePrice, very high), (To...","((OverallCond, 5), (YearRemodAdd, very high))"
"{('BldgType', '1Fam'), ('OverallCond', 5), ('TotalBsmtSF', 'very high'), ('YearRemodAdd', 'very high')} -> {('SalePrice', 'very high'), ('YearBuilt', 'very high')}",4,2,86,134,0.050000,0.848837,9.248525,0.042442,6.008219,"((BldgType, 1Fam), (OverallCond, 5), (TotalBsm...","((SalePrice, very high), (YearBuilt, very high))"


##### We can filter rules with high/very high sale price:

In [176]:
def is_in_itemset(itemset, key,value):
    if type(value) is not list:
        value = [value]
    for (k,v) in itemset:
            if k ==key and v in value:
                return True
    return False
    

In [191]:
filter_rhs = rules_df.rhs.apply(lambda x: is_in_itemset(x,'SalePrice',['high','very high']))

In [196]:
rules_df[filter_rhs].sort_values(['len_r','support'],ascending=[True,False])

,len_l,len_r,count_lhs,count_rhs,support,confidence,lift,rpf,conviction,lhs,rhs
rule,,,,,,,,,,,
"{('BldgType', '1Fam'), ('OverallQual', 8)} -> {('SalePrice', 'very high')}",2,1,142,289,0.080822,0.830986,4.198060,0.067162,4.745491,"((BldgType, 1Fam), (OverallQual, 8))","((SalePrice, very high),)"
"{('BldgType', '1Fam'), ('FullBath', 2), ('OverallQual', 8)} -> {('SalePrice', 'very high')}",3,1,128,289,0.073288,0.835938,4.223075,0.061264,4.888715,"((BldgType, 1Fam), (FullBath, 2), (OverallQual...","((SalePrice, very high),)"
"{('BldgType', '1Fam'), ('OverallCond', 5), ('OverallQual', 8)} -> {('SalePrice', 'very high')}",3,1,122,289,0.071233,0.852459,4.306540,0.060723,5.436149,"((BldgType, 1Fam), (OverallCond, 5), (OverallQ...","((SalePrice, very high),)"
"{('TotalBsmtSF', 'very high'), ('YearBuilt', 'very high')} -> {('SalePrice', 'very high')}",2,1,120,289,0.067123,0.816667,4.125721,0.054817,4.374844,"((TotalBsmtSF, very high), (YearBuilt, very hi...","((SalePrice, very high),)"
"{('BldgType', '1Fam'), ('FullBath', 2), ('OverallCond', 5), ('OverallQual', 8)} -> {('SalePrice', 'very high')}",4,1,114,289,0.065753,0.842105,4.254234,0.055371,5.079680,"((BldgType, 1Fam), (FullBath, 2), (OverallCond...","((SalePrice, very high),)"
"{('OverallCond', 5), ('TotalBsmtSF', 'very high'), ('YearBuilt', 'very high')} -> {('SalePrice', 'very high')}",3,1,117,289,0.065068,0.811966,4.101973,0.052833,4.265473,"((OverallCond, 5), (TotalBsmtSF, very high), (...","((SalePrice, very high),)"
"{('BldgType', '1Fam'), ('TotalBsmtSF', 'very high'), ('YearBuilt', 'very high')} -> {('SalePrice', 'very high')}",3,1,110,289,0.060959,0.809091,4.087449,0.049321,4.201239,"((BldgType, 1Fam), (TotalBsmtSF, very high), (...","((SalePrice, very high),)"
"{('FullBath', 2), ('TotalBsmtSF', 'very high'), ('YearBuilt', 'very high')} -> {('SalePrice', 'very high')}",3,1,109,289,0.060274,0.807339,4.078601,0.048662,4.163046,"((FullBath, 2), (TotalBsmtSF, very high), (Yea...","((SalePrice, very high),)"
"{('BldgType', '1Fam'), ('OverallCond', 5), ('TotalBsmtSF', 'very high'), ('YearBuilt', 'very high')} -> {('SalePrice', 'very high')}",4,1,107,289,0.058904,0.803738,4.060408,0.047343,4.086660,"((BldgType, 1Fam), (OverallCond, 5), (TotalBsm...","((SalePrice, very high),)"
